In [2]:
import pathlib
basedir = pathlib.Path.cwd().parent
deviceId = '0'
pathClf = basedir / 'data' / 'processed' / f'calibrated_classifier_{"DEBERTA"}_{"95perc"}.pkl'
pathPCA = basedir / 'data' / 'processed' / f'pca_projector_{"DEBERTA"}_{"95perc"}.pkl'


In [3]:
import os
# Set environment variables
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = deviceId
os.environ["WORLD_SIZE"] = "1"
import joblib
import torch
# from transformers import DistilBertTokenizer, DistilBertModel
from transformers import DebertaV2Tokenizer, DebertaV2Model


class StanceClassifier:
    def __init__(self, pathPCA, pathClf, device):
        self.device = device
        # Load the text embedder (DistilBERT)
        self.tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v2-xlarge')
        self.model = DebertaV2Model.from_pretrained('microsoft/deberta-v2-xlarge').to(device)
        # Load the trained classifier and PCA projector
        self.calibrated_clf = joblib.load(pathClf)
        self.pca = joblib.load(pathPCA)

    def get_embeddings(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().squeeze().numpy()

    def predict_proba(self, text):
        # Extract embeddings using DistilBERT
        embedding = self.get_embeddings(text).reshape(1, -1)
        
        # Compress embedding using trained PCA
        embedding_pca = self.pca.transform(embedding)
        
        # Compute probability score using trained calibrated logistic regression
        prob_score = self.calibrated_clf.predict_proba(embedding_pca)[:, 1][0]
        
        return prob_score



In [4]:
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = StanceClassifier(pathPCA=pathPCA, pathClf=pathClf, device=device)


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

/home/coppolillo/venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/coppolillo/venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator IsotonicRegression from version 1.5.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/coppolillo/venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CalibratedClassifierCV from version 1.5.1 w

In [5]:
example_text = "Brexit? I support Nigel Farage and UKIP reasons to leave the EU market"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')


Predicted probability score: 0.0267


In [6]:
example_text = "I will vote for leave but I like Schengen free travel area"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')


Predicted probability score: 0.1784


In [50]:
example_text = "I am half between remain and leave"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')



Predicted probability score: 0.5708


In [54]:
example_text = "I strongly support Remain! Brexit will destroy the UK economy"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')



Predicted probability score: 0.6368


In [55]:
example_text = "I vote Remain because I am an immigrant and I want to feel welcomed."
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')


Predicted probability score: 0.7305


In [15]:
example_text = "I vote Remain because I am an immigrant and I want to feel welcomed. #StrongerIn"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')


Predicted probability score: 0.9913


In [20]:
example_text = "Brexit is the most disastrous policy of all time, and history is repeating itself. Is it not true that this can be the case?"
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {probability_score:.4f}')


Predicted probability score: 0.4359


In [14]:
example_text

'Brexit is the most disastrous policy of all time, and history is repeating itself. Is it not true that this can be the case?'

In [10]:
from transformers import (
    Adafactor,
    AutoTokenizer,
    LlamaTokenizer,
    HfArgumentParser,
    pipeline
)

In [11]:
sentiment_model = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb")

In [16]:
sentiment_outputs = sentiment_model(example_text)
messages_values = [output["score"] if output["label"] == "POSITIVE" else 1 - output["score"]
                   for output in sentiment_outputs]

In [17]:
messages_values

[0.6802294254302979]

In [22]:
example_text = "Brexit is bullshit"

In [24]:
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {1 - probability_score:.4f}')

Predicted probability score: 1.0000


In [25]:
sentiment_outputs = sentiment_model(example_text)
[output["score"] if output["label"] == "POSITIVE" else 1 - output["score"]
                   for output in sentiment_outputs]

[0.028895556926727295]

In [26]:
example_text = "Brexit is Heaven"

In [27]:
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {1 - probability_score:.4f}')

Predicted probability score: 1.0000


In [28]:
sentiment_outputs = sentiment_model(example_text)
[output["score"] if output["label"] == "POSITIVE" else 1 - output["score"]
                   for output in sentiment_outputs]

[0.9247830510139465]

## 1: PRO-Brexit, 0: AGAINST-Brexit

In [29]:
example_text = "Brexit is not good for economy"

In [30]:
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {1 - probability_score:.4f}')

Predicted probability score: 0.7284


In [31]:
sentiment_outputs = sentiment_model(example_text)
[output["score"] if output["label"] == "POSITIVE" else 1 - output["score"]
                   for output in sentiment_outputs]

[0.03201204538345337]

In [32]:
example_text = "Brexit is not bad for economy"

In [33]:
probability_score = classifier.predict_proba(example_text)
print(f'Predicted probability score: {1 - probability_score:.4f}')

Predicted probability score: 0.7284


In [34]:
sentiment_outputs = sentiment_model(example_text)
[output["score"] if output["label"] == "POSITIVE" else 1 - output["score"]
                   for output in sentiment_outputs]

[0.8435317873954773]